Citation: Notebook used as reference guide for fine-tuning Gemma 2B models
https://colab.research.google.com/github/adithya-s-k/LLM-Cookbook/blob/main/Finetuning/Gemma_finetuning_notebook.ipynb

In [ ]:
!pip install -q -U bitsandbytes==0.42.0
!pip install -q -U peft==0.8.2 # for parameter efficient fine-tuning
!pip install -q -U trl==0.7.10
!pip install -q -U accelerate==0.27.1
!pip install -q -U datasets==2.17.0
!pip install -q -U transformers==4.38.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━

In [ ]:
!pip install transformers[cohere]

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model_name = 'google/gemma-2b-it' # we use the google gemma 2B instruct model

# we use the 4 bit quantized version to perform qLora
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd outputs

In [ ]:
cd '/content/drive/MyDrive/CS 224N Dataset'

In [ ]:
cd ..

In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="MALLS-v0.1-train.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
def prompt(row):
  text = f"<start_of_turn>user Convert the following natural language statement to a first-order logic notation: {row['NL']} <end_of_turn>\n<start_of_turn>model {row['FOL']} <end_of_turn>"
  return text


prompt_column = [prompt(row) for row in dataset]
dataset = dataset.add_column("prompt", prompt_column)

In [ ]:
dataset = dataset.shuffle(seed=1234)
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)
dataset = dataset.train_test_split(test_size=0.4)
train_data = dataset["train"]
test_data = dataset["test"]

Map:   0%|          | 0/27284 [00:00<?, ? examples/s]

In [ ]:
train_data[0]

{'FOL': '∀x (BusinessStrategy(x) ∧ (IncreasesRevenue(x) ∨ ReducesCosts(x)) ∧ ¬HarmsReputation(x) ∧ ¬ResultsInLegalTroubles(x) → Successful(x))',
 'NL': "A business strategy is successful if it increases revenue or reduces costs, but it is not successful if it harms the company's reputation or results in legal troubles.",
 'prompt': "<start_of_turn>user Convert the following natural language statement to a first-order logic notation: A business strategy is successful if it increases revenue or reduces costs, but it is not successful if it harms the company's reputation or results in legal troubles. <end_of_turn>\n<start_of_turn>model ∀x (BusinessStrategy(x) ∧ (IncreasesRevenue(x) ∨ ReducesCosts(x)) ∧ ¬HarmsReputation(x) ∧ ¬ResultsInLegalTroubles(x) → Successful(x)) <end_of_turn>",
 'input_ids': [2,
  106,
  1645,
  25151,
  573,
  2412,
  4158,
  5255,
  6218,
  577,
  476,
  1370,
  235290,
  2308,
  14855,
  36932,
  235292,
  586,
  2567,
  9803,
  603,
  8385,
  1013,
  665,
  12062

In [ ]:
import pandas as pd

df = pd.DataFrame(train_data)
NL_list = df['NL'].tolist()
true_FOL = df['FOL'].tolist()
predicted_FOL = []

def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  <start_of_turn>user
  Convert the following natural language statement to a first-order logic notation. JUST OUTPUT THE FIRST ORDER LOGIC STATEMENT:
  {query}
  <end_of_turn>\n<start_of_turn>model


  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=150, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  # decoded = tokenizer.batch_decode(generated_ids)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return (decoded)

for i in range(0, len(NL_list)):  # Batch size of 8
  prediction = get_completion(NL_list[i], model, tokenizer)
  print(prediction)
  predicted_FOL.append(prediction)



A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

KeyboardInterrupt: 

In [ ]:
cd '/content/drive/MyDrive/CS 224N Dataset'

/content/drive/MyDrive/CS 224N Dataset


In [ ]:
cd outputs

/content/drive/MyDrive/CS 224N Dataset/outputs


In [ ]:
import json

# Function to clean and extract FOL statements
def extract_fol_statements(entries):
    cleaned_data = []
    for entry in entries:
        start = entry.find('model\n\n\n') + len('model\n\n\n')
        if start > len('model\n\n\n') - 1:  # Check if 'model\n\n\n' was found
            cleaned_data.append(entry[start:].strip())
    return cleaned_data

# Clean the predicted FOL statements
cleaned_predicted_FOL = extract_fol_statements(predicted_FOL)

# Combine NL_list, cleaned_predicted_FOL, and true_FOL into a single list of dictionaries
combined_data = [
    {"NL": nl, "predicted_FOL": pred_fol, "true_FOL": true_fol}
    for nl, pred_fol, true_fol in zip(NL_list, cleaned_predicted_FOL, true_FOL)
]

# Convert the combined data to a JSON object
json_object = json.dumps(combined_data, indent=4)

# Write the JSON object to a file
with open('combined_data_vanilla_model.json', 'w') as f:
    f.write(json_object)

print("Combined data has been saved to 'combined_data.json'")
